In [3]:
from __future__ import unicode_literals

import json

from hazm import *


def read_file():
    documents_title = []
    documents_content = []
    documents_url = []
    with open('../Phase_1/assets/IR_data_news_12k.json', encoding='UTF-8') as f:
        data = json.load(f)
        for i in data:
            documents_title.append(data[i]["title"])
            documents_content.append(data[i]["content"])
            documents_url.append(data[i]['url'])
    return documents_url, documents_title, documents_content

docs_url, docs_title, docs_content = read_file()
normalizer = Normalizer()

In [4]:
# coding: utf8
from os import path
import codecs
from hazm.Normalizer import Normalizer
from Phase_1.src.utils import data_path

default_stop_words = path.join(data_path, 'stopwords.dat')


class StopWord:
    """ Class for remove stop words

         >>> StopWord().clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران', 'کی']
         >>> StopWord(normal=True).clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران']

         """

    def __init__(self, file_path=default_stop_words, normal=False):
        self.file_path = file_path
        self.normal = normal
        self.normalizer = Normalizer().normalize
        self.stop_words = self.init(file_path, normal)

    def init(self, file_path, normal):
        if not normal:
            return set(
                line.strip("\r\n") for line in codecs.open(file_path, "r", encoding="utf-8").readlines())
        else:
            return set(
                self.normalizer(line.strip("\r\n")) for line in
                codecs.open(file_path, "r", encoding="utf-8").readlines())

    def set_normalizer(self, func):
        self.normalizer = func
        self.stop_words = self.init(self.file_path, self.normal)

    def __getitem__(self, item):
        return item in self.stop_words

    def __str__(self):
        return str(self.stop_words)

    def clean(self, iterable_of_strings, return_generator=False):
        if return_generator:
            return filter(lambda item: not self[item], iterable_of_strings)
        else:
            return list(filter(lambda item: not self[item], iterable_of_strings))


In [5]:
class Document:
    def __init__(self, content, url, title):
        self.content = content
        self.url = url
        self.title = title


In [6]:
def preprocess_content(content):
    str_empty = ' '
    content = normalizer.normalize(content)
    content = Stemmer().stem(content)
    content = word_tokenize(content)
    content = StopWord(normal=False).clean(content)
    return str_empty.join(content)


def process_positions(preprocessed_content):
    positions = {}
    for position_of_word, word in enumerate(preprocessed_content):
        if word not in positions:
            positions[word] = []
        positions[word].append(position_of_word)
    return positions


def generate_index(positions, title):
    return {"number of occurrences in document": len(positions),
            "positions": positions,
            "title of document": title}


class PositionalIndex:
    def __init__(self, documents_url, documents_title, documents_content):
        self.Documents = [Document(documents_content[i], documents_url[i], documents_title[i]) for i in
                          range(len(documents_url))]
        self.positional_index_structure = {}
        self.build_positional_index()

    def build_positional_index(self):
        for i, document in enumerate(self.Documents):
            preprocessed_content = preprocess_content(document.content)
            processed_content = process_positions(preprocessed_content)
            title = document.title
            url = document.url
            for word, positions in processed_content.items():
                self.add_index(word, positions, title, url)

    def add_index(self, word, positions, title, url):
        index_to_add = generate_index(positions, title)
        if word not in self.positional_index_structure:
            self.positional_index_structure[word] = {"total occurrences": 0,
                                                     "indexes": {}}
        self.positional_index_structure[word]["total occurrences"] += index_to_add[
            "number of occurrences in document"]
        self.positional_index_structure[word]["indexes"][url] = index_to_add


In [7]:
def calc_string(string):
    state = False
    i = 0
    while i < len(string):
        ch = string[i]
        if ch == "\"":
            if state:
                string = string[:i] + '>' + string[i + 1:]
            else:
                string = string[:i] + '<' + string[i + 1:]
            state = 1 ^ state
        if ch == '!':
            print(f'ch is {ch}, i is {i}, string[i] is {string[i]}, string is {string}')
            string = string[:i - 1] + ' NOT' + string[i + 1:]
        i += 1
    return string


def preprocess_query(query):
    query = calc_string(query)
    str_empty = ' '
    query = normalizer.normalize(query)
    query = Stemmer().stem(query)
    query = word_tokenize(query)
    query = StopWord(normal=False).clean(query)
    query = str_empty.join(query)
    return query


def docID(plist, number):
    lst_urls = sorted(list(plist.keys()))
    return lst_urls[number]


def position(plist):
    return plist['positions']


def pos_intersect(p1, p2, k):
    answer = {}  # answer <- ()
    len1 = len(p1)
    len2 = len(p2)
    i = j = 0
    while i != len1 and j != len2:
        key = docID(p1, i)
        key2 = docID(p2, j)
        if key == key2:
            l = []
            pp1 = position(p1[key])
            pp2 = position(p2[key])

            plen1 = len(pp1)
            plen2 = len(pp2)
            ii = jj = 0
            while ii != plen1:
                while jj != plen2:
                    if abs(pp1[ii] - pp2[jj]) <= k:
                        l.append(pp2[jj])
                    elif pp2[jj] > pp1[ii]:
                        break
                    jj += 1
                #l.sort()
                while l != [] and abs(l[0] - pp1[ii]) > k:
                    l.remove(l[0])
                for ps in l:
                    if key in answer:
                        # answer[key]['positions'].extend([pp1[ii], ps])
                        answer[key]['positions'].extend([max(pp1[ii], ps)])
                    else:
                        answer[key] = {'positions': [max(pp1[ii], ps)]}
                ii += 1
            i += 1
            j += 1
        elif key < key2:
            i += 1
        else:
            j += 1
    return answer


In [8]:
pos_index = PositionalIndex(documents_url=docs_url, documents_title=docs_title, documents_content=docs_content)

In [9]:
class QueryHandler:
    def __init__(self, positional_index):
        self.positional_index = positional_index

    def answer_query(self, query):
        query = preprocess_query(query)
        not_queries = []
        and_queries = []
        empty_str = ' '
        is_not = False
        consecutive = []

        i = 0
        while i < len(query):
            next_occur = query.find(' ', i)
            if next_occur < i:
                break
            substr = query[i:next_occur]
            if substr == 'NOT':
                is_not = True
                i = next_occur + 1
                continue
            elif is_not:
                not_queries.append(self.get_result([substr]))
                is_not = False
            elif substr[0] == '>':
                consecutive.append(substr[1:])
                i = next_occur + 1
                continue
            elif substr[-1] == '<':
                consecutive.append(substr[:-1])
                and_queries.append(self.get_result(consecutive))
                i = next_occur + 1
                consecutive.clear()
                continue
            and_queries.append(self.get_result([substr]))
            i = next_occur + 1
            print(i)
        #
        and_results = self.handle_queries(and_queries)
        not_results = self.handle_queries(not_queries)
        result = self.remove_by_sub_queries(and_results, not_results)
        return result

    def remove_by_sub_queries(self, in_results, out_results):
        result = []
        for in_item_url in in_results:
            if in_item_url in out_results:
                continue
            index = {'title': self.positional_index.positional_index_structure[in_item_url]['title'],
                     'score': len(in_results[in_item_url]['positions']),
                     'url': in_item_url}
            result.append(index)
        return result

    @staticmethod
    def handle_queries(queries):
        if not queries:
            return {}
        result = queries[0]
        for item_to_intersect in queries[1:]:
            result = pos_intersect(result, item_to_intersect, 1000000)
        return result

    def get_result(self, words):
        if words[0] not in self.positional_index.positional_index_structure:
            return {}
        result = self.positional_index.positional_index_structure[words[0]]['indexes']
        for word in words[1:]:
            if word not in self.positional_index.positional_index_structure:
                result = {}
                break
            indexes_of_word = self.positional_index.positional_index_structure[word]['indexes']
            result = pos_intersect(result, indexes_of_word, 1)
        return result


query_handler = QueryHandler(positional_index=pos_index)
query_handler.answer_query('تحریم های آمریکا علیه ایران')

10
17


[]

['اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'سجادی :حضور تماشاگران در  لیگ برتر فوتبال تابع نظر فدراسیون  و سازمان لیگ است', 'محل برگزاری نشست\u200cهای خبری سرخابی\u200cها؛ مجیدی در سازمان لیگ، گل\u200cمحمدی در تمرین پرسپولیس', 'ماجدی در نشست با صالحی امیری: امیدوارم در این دوره تیم  المپیک موفق شود', 'لیگ\u200cبرتر بسکتبال|\u200c نخستین پیروزی شهرداری گرگان و ذوب\u200cآهن در پلی\u200cآف', 'مسابقات تنیس روی میز فیدر قطر| هر4 بانوی ملی پوش ایران حذف شدند', 'اعلام برنامه نشست خبری گل محمدی/ مجیدی هم باید به محل تمرین پرسپولیس برود!', 'احضار مدیران پرسپولیس به کمیته انضباطی  پیش از دربی', 'مدیر بی\u200cدستاورد، رئیس شد/ روزگار تلخ\u200cتر از تلخ برای والیبال ارومیه؟', 'خبر خوب برای استقلال؛ دانشگر با تیم تمرین کرد', 'تراکتور در آستانه بازگشت به خانه اصلی/ نواقص ورزشگاه یادگار امام بررسی شد', 'تورنمنت شش جانبه تایلند| پیروزی پر گل  فوتسالیست\u200cهای  جوان ایران برابر مغولستان', 'گزارش تمرین پرسپولیس| روحیه بالای شاگردان گل محمدی در آستانه دربی', 'پولادگر: درخشش\u200cهای هندبال مایه